In [152]:
import matplotlib.pyplot as plt
import pathlib
import requests
import io
import os
import numpy as np
import pathlib
import PIL
import base64

import tensorflow as tf
import cv2        as cv
import numpy      as np

from fastapi                  import FastAPI,File, UploadFile
from fastapi.middleware.cors  import CORSMiddleware
#from mushroom_learning.gcp    import get_model

from tensorflow               import keras
from tensorflow.keras         import utils


# Checking API

In [153]:
URL_base    = 'https://mushroom-docker-lpuaioudtq-ew.a.run.app/'
URL_base    = 'http://127.0.0.1:8000/'
URL_size    = URL_base+"size/"
URL_predict = URL_base+"predict/"

print(URL_size)
print(URL_predict)

http://127.0.0.1:8000/size/
http://127.0.0.1:8000/predict/


## Open an Image

To really test how it will be in production, we need to put this notebook somewhere it can see the image, but the api cannot.

In [154]:
!ls -rtlh 'amanita.jpg'
!pwd

-rwxr-xr-x 1 danielsptanner danielsptanner 85K Mar  9 11:35 amanita.jpg
/home/danielsptanner/code/DSP-Tan/mushroom_learning/notebooks


In [155]:
amanita_loc='amanita.jpg'
image=PIL.Image.open(amanita_loc)
print(type(image))
image_array = np.array(image)
print(image_array.shape)
print(f'So every row has 1024 columns, each with a depth of 3, total elements:{657*1024*3}')
print(f'\nThese are not bytes, as this does not bear a fixed relation to the filesize: {2018304/59836}')

<class 'PIL.JpegImagePlugin.JpegImageFile'>
(600, 800, 3)
So every row has 1024 columns, each with a depth of 3, total elements:2018304

These are not bytes, as this does not bear a fixed relation to the filesize: 33.73059696503777


## Check uvicorn run locally

### Check base URL

In [156]:
URL_base    = 'http://127.0.0.1:8000/'
URL_size    = URL_base+"size/"
URL_predict = URL_base+"predict/"

In [158]:
response = requests.get(URL_base)
print(response.json())

["Don't eat that mushroom!"]


### Check image size

#### Open image as binary, write to np array byte array, and encode as base64

In [159]:
! ls -rtlh amanita.jpg 
with open('amanita.jpg', 'rb') as f:
    im_API = f.read()
im_API=np.asarray(bytearray(im_API), dtype="uint8")
encoded = base64.b64encode(im_API)

-rwxr-xr-x 1 danielsptanner danielsptanner 85K Mar  9 11:35 amanita.jpg


#### Confirm we can decode this and repeat preprocessing steps

In [160]:
decoded_mush=base64.decodebytes(encoded)
print(f'filesize {len(decoded_mush)/1000}, type of array {type(decoded_mush)}')
new_im_API=np.asarray(bytearray(decoded_mush), dtype="uint8")
np.array_equal(new_im_API,im_API)

filesize 86.717, type of array <class 'bytes'>


True

### Pass encoded array to API /size end point

In [161]:
files={'mush':encoded}
response = requests.get(URL_size,data=files)
print(response.json())

This file is 86.717 Kbytes and type <class 'bytes'>


### Pass encoded array to API /predict end point

In [163]:
files={'mush':encoded}
response = requests.get(URL_predict,data=files)
print(response)
print(response.json())

<Response [500]>


JSONDecodeError: [Errno Expecting value] Internal Server Error: 0

In [164]:
! pwd
model=keras.models.load_model('../model_species_vgg_v1/')

/home/danielsptanner/code/DSP-Tan/mushroom_learning/notebooks


OSError: No file or directory found at ../model_species_vgg_v1/

In [24]:
def check_size(mush: bytes = File(...)):
    # convert to bytes with bytearray, and to np array
    image = np.asarray(bytearray(mush), dtype="uint8")
    return f'This file is {len(image)/1000} Kbytes'

In [49]:
img = PIL.Image.open(amanita_loc, mode='r')
print(type(img))
img_byte_arr = io.BytesIO()
img.save(img_byte_arr, format='jpeg')
img_byte_arr = img_byte_arr.getvalue()
print(len(img_byte_arr)/1000)
print(check_size(img_byte_arr))

<class 'PIL.JpegImagePlugin.JpegImageFile'>
78.026
This file is 78.026 Kbytes


In [46]:
!ls -rtlh amanita.jpg

-rwxr-xr-x 1 danielsptanner danielsptanner 77K Mar  9 07:21 amanita.jpg


In [71]:
amanita_loc='amanita.jpg'
image=PIL.Image.open(amanita_loc)
files={'mush':image}
response = requests.get(URL_size,data=files)
print(response.json())

This file is 0.082 Kbytes


In [70]:
! ls -rtlh amanita.jpg

-rwxr-xr-x 1 danielsptanner danielsptanner 77K Mar  9 07:21 amanita.jpg


In [ ]:
files={'mush':im_API}

In [127]:
files = {'mush': open('/home/danielsptanner/APE2017-9186528_HyZ-iZy6vg.JPG', 'rb')}
response = requests.get(URL_size,data=files)
print(response.json())

This file is 0.575 Kbytes


## Check prediction

In [83]:
files = {'mush': image_path}
response = requests.get(URL_predict,data=files)
print(response.json())

This mushroom is most likely poisonous. Score: 0.64


### Check prediction against local model.

In [87]:
def image_API(im_path,size):
    # convert to bites (this step should be excluded in an actual API, as the
    # image arrives in bytes from the get or post functions.)
    with open(im_path, 'rb') as f:
        im_API = f.read()
    # convert to np array.    
    im_API=np.asarray(bytearray(im_API), dtype="uint8")
    # decode byte array back into image, and then adjust
    # for cv's automatic BGR representation
    im_API = cv.imdecode(im_API,cv.IMREAD_COLOR)
    im_API = cv.cvtColor(im_API , cv.COLOR_BGR2RGB)
    
    # resize using tensor flow with nearest neighbour interpolation
    im_API=tf.image.resize(im_API,size, method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    # Expand for CNN
    im_API = tf.expand_dims(im_API, 0) 
    
    return im_API

In [94]:
im_API=image_API(image_path,(224,224))
model=get_model()

../our_first_model


In [96]:
prediction_API=model.predict(im_API)
classif_API = int(prediction_API > .5)
print(f'{prediction_API[0][0]:.5f}:{("poison","edible")[classif_API]}')

0.63804:edible
